In [2]:
import pandas as pd
good = pd.read_csv('data/good.tsv', sep='\t')

,context_2,context_1,context_0,reply,label
32219,эй . не трогай его !,разве ты не достаточно натворил ?,я серьезно . . . прочь с моей земли .,и я тоже . больше никаких глупостей .,good
25205,NaN,косима !,привет .,"как дела , дружище ?",good
38601,нужно приготовить завтрак .,а это еще от куда ?,"джонатан , сегодня ночью мне снились невероятн...",это был не сон .,good


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(good.context_0)
matrix_big = vectorizer.transform(good.context_0)


(60049, 14123)

In [6]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300)
svd.fit(matrix_big)
matrix_small = svd.transform(matrix_big)


(60049, 300)

In [7]:
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

def softmax(x):
    probe = np.exp(x)
    return probe/sum(probe)

class NeighbourSampler(BaseEstimator):
    def __init__(self, k=5, temperature = 1.0):
        self.k = k
        self.temperature = temperature
    def fit(self, x, y):
        self.tree_ = BallTree(x)
        self.y_ = np.array(y)
    def predict(self, x, random_state=None):
        distances, indices = self.tree_.query(x, return_distance = True, k=self.k)
        result = []
        for distance, index in zip(distances, indices):
            result.append(np.random.choice(index, p = softmax(distance * self.temperature)))
            return self.y_[result]

In [10]:
from sklearn.pipeline import make_pipeline
ns = NeighbourSampler()
ns.fit(matrix_small, good.reply)
pipe = make_pipeline(vectorizer, svd, ns)

array(['отличный выбор .'], dtype=object)

In [16]:
import telebot
TOKEN = ''
bot = telebot.TeleBot(TOKEN)
@bot.message_handler(commands=['start', 'help'])
def say_hi(message):
    bot.reply_to(message, "Hi! How's that goin'?")
@bot.message_handler(func=lambda message: True)
def echo_all(message):
    bot.reply_to(message, pipe.predict([message.text.lower()])[0])
bot.polling()